In [1]:
import os
import sys
import shutil
import pickle
import numpy as np
import pandas as pd
import datetime
from pathlib import Path
from tqdm import tqdm

from concorde.tools import ascii_replace
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from shapely.geometry import Point
import geopandas as gpd

In [2]:
path = Path(r'../data/tracks/climada')
with open(path/'tracksNC_historicalStorms.pkl', 'rb') as fin:
    dctTracks = pickle.load(fin)

In [3]:
dctTracks.keys()

dict_keys(['ISABEL', 'IRENE', 'ARTHUR', 'MATTHEW', 'FLORENCE', 'DORIAN', 'ISAIAS'])

In [4]:
mask = gpd.read_file(r'../gis/gpkg/polygon_coastline_NC.gpkg')
def radImpact(vmax, rmax, vr = 33, beta = 1):
    r1 = rmax
    if rmax == 0:
        return 1
    else:
        while True:
            r = rmax * (vr / vmax)**(-2 / beta) * (np.exp(1 - (rmax / r1)**beta))**(1 / beta)
            if np.abs(r - r1) < 1e-5:
                break
            r1 = r
        if r == 0:
            return rmax
        else:
            return r

In [11]:
dctTracksShorted = {}
for k in tqdm(dctTracks.keys()):
    aux = dctTracks[k]
    ri = [radImpact(aux.loc[i, 'max_sustained_wind']/1.94384, 
                    aux.loc[i, 'radius_max_wind']) for i in aux.index]
    inter = []
    for i, r in zip(aux.index, ri):
        pnt = Point((aux.loc[i, 'lon'], aux.loc[i, 'lat']))
        pol = pnt.buffer(r/110)
        if mask.iloc[0].geometry.intersects(pol):
            inter.append(1)
        else:
            inter.append(0)
    aux['windIntersectNC'] = inter
    indEnter = list(aux['windIntersectNC']).index(1) ## 1st time step with intersection
    indStart = max(indEnter - 4*8, 0) ## 4 days and 8 time steps per day
    indLeave = len(aux) - 1 - list(aux['windIntersectNC'])[::-1].index(1)
    indFinish = min(indLeave + 1*8, len(aux) - 1) ## 1 day and 8 time steps per day
    auxShort = aux.iloc[indStart:indFinish, :]
    aux = aux.drop(['windIntersectNC'], axis = 1)
    dctTracksShorted[f'{k}'] = aux.loc[auxShort.index, :]

100%|██████████| 7/7 [00:00<00:00, 79.13it/s]


##### Setup simulations

Function to get the random tide was deleted by accident

In [12]:
def getRandomTide(dfs, k):
    ''' updated function 01/05/2023 only one storm as input
    '''
    # dfs = df[df['tc_number'] == tc]
    dur = (dfs['randomDate'].iloc[-1] - dfs['randomDate'].iloc[0]).total_seconds()/86400 + 3
    hh = dfs['randomDate'].iloc[0].hour
    dd = dfs['randomDate'].iloc[0].day
    mm = dfs['randomDate'].iloc[0].month
    yy = dfs['randomDate'].iloc[0].year
    with open(r'TideFac-Code-Executable/dates.txt', 'w') as fout:
        fout.write(f'{dur:0.3f}\n')
        fout.write(f'{hh:02d},{dd},{mm},{yy}')
    os.system(r'TideFac-Code-Executable/a.out < TideFac-Code-Executable/dates.txt > TideFac-Code-Executable/screen.txt')
    
    aux = pd.read_csv('tide_fac.out', skiprows=9, header = None, delim_whitespace = True,
                           names = [f'node_factor_{k}', f'eq_arg_{k}'])
    
    return aux

In [13]:
pathout = Path(r'../models/adcirc/concorde/batch02/02c')
pathtemplate = pathout/'_template'

In [14]:
for itc, tc in tqdm(enumerate(dctTracks.keys())):

    dfs1 = dctTracksShorted[tc] ## shortened track
    dfs1 = dfs1[['lon', 'lat', 'max_sustained_wind', 'central_pressure', 'radius_max_wind']]
    dfs1['randomDate'] = dfs1.index
    dfs1.index = range(len(dfs1))
    dfr = pd.DataFrame(np.broadcast_to(dfs1.iloc[0, :].values, (8*3 + 1, len(dfs1.columns))),
                    columns = dfs1.columns)
    ## ramp period date, starts from first timesteps going backwards 3 days
    rampDates = [dfs1['randomDate'].iloc[0] - i*pd.Timedelta(hours = 3) for i in range(1, len(dfr)+1)][::-1]
    dfr['randomDate'] = rampDates
    dfr['central_pressure'] = [1013] * len(dfr) ## atmospheric pressure
    dfr['max_sustained_wind'] = [1] * len(dfr) ## 1knot wind speed

    date_aux0 = dfr['randomDate'].iloc[0]
    if date_aux0.hour == 0:
        dfr2 = dfr.copy()
    else:
        ## df ramp is extended until the starting hour is 00
        date_aux1 = datetime.datetime(date_aux0.year, date_aux0.month, date_aux0.day, 0)
        r_extra = pd.date_range(start = str(date_aux1), end = str(date_aux0), freq = '3H')
        dfr_extra = dfr.iloc[0:len(r_extra)-1, :]
        dfr_extra['randomDate'] = r_extra[:-1]
        dfr2 = pd.concat([dfr_extra, dfr])
        dfr2.index = range(len(dfr2))

    dfe = pd.concat([dfr2, dfs1], ignore_index = True)
    dftides = getRandomTide(dfe, tc)

    namerun = tc

    try:
        os.mkdir(pathout/namerun)
    except:
        pass

    ## write fort22
    with open(pathout/namerun/'fort.22', 'w') as fout:
        for i in dfe.index:
            d = dfe.loc[i, 'randomDate'].strftime('%Y%m%d%H')
            inc = int((dfe.loc[i, 'randomDate'] - dfe['randomDate'].iloc[0]).total_seconds()/(3600))
            lat = int(np.round(dfe.loc[i, 'lat'], 1)*10)
            lon = int(np.abs(np.round(dfe.loc[i, 'lon'], 1))*10)
            ws = int(np.round(dfe.loc[i, 'max_sustained_wind']*1.94384, 0)) ## meter per second to knots
            p = int(np.round(dfe.loc[i, 'central_pressure'], 0))
            rmw = int(np.round(dfe.loc[i, 'radius_max_wind'], 0))
            text = (
                    f'AL, 00, {d},   , BEST, {inc:3},{lat:4}N, {lon:4}W, '
                    f'{ws:3}, {p:4},   ,  00, XXX,  000,  000,   00,  000, 0000,     ,'
                    f' {rmw:3},     ,    ,    ,    ,    ,000,  00,  {tc}  ,  00,    0, 0, 0, 0, 0,     00.0,'
                    f'   00.0,   00.0,   00.0,    0.0000,   0.0000,   0.0000,   0.0000,   0.0000,'
                    f' 000.0000, 000.0000, 000.0000, 000.0000\n')
            fout.write(text)

    pathout_sub = pathout/namerun
    
    ## modify fort.15
    filein = pathtemplate/'fort.15'
    fileout = pathout_sub/'fort.15'
    olds = ['XX1', 'XX2', 'YYYY_MM_DD_HHmm', 'DDD', 'RRR',
            'M2NODF1', 'M2EQA1', 'S2NODF1', 'S2EQA1', 'N2NODF1', 'N2EQA1', 'K2NODF1', 'K2EQA1', 'K1NODF1', 'K1EQA1', 
            'O1NODF1', 'O1EQA1', 'P1NODF1', 'P1EQA1', 'Q1NODF1', 'Q1EQA1',
            'M2NODF2', 'M2EQA2', 'S2NODF2', 'S2EQA2', 'N2NODF2', 'N2EQA2', 'K2NODF2', 'K2EQA2', 'K1NODF2', 'K1EQA2', 
            'O1NODF2', 'O1EQA2', 'P1NODF2', 'P1EQA2', 'Q1NODF2', 'Q1EQA2',
            'datedate']

    dftides_sub = dftides.loc[['M2', 'S2', 'N2', 'K2', 'K1', 'O1', 'P1', 'Q1'], [x for x in dftides.columns if x.endswith(f'_{tc}')]]
    constituents = [f'{x:.5f}' for x in dftides_sub.values.reshape(-1)]
    dur = (dfe['randomDate'].iloc[-1] - dfe['randomDate'].iloc[0]).total_seconds()/86400

    dramp = (dfr2['randomDate'].iloc[-1] - dfr2['randomDate'].iloc[0]).total_seconds()/86400
    news = [f'{itc:04d}', f'STORM{tc}', dfe['randomDate'].iloc[0].strftime('%Y %m %d %H00'), str(dur), str(dramp)]
    news.extend(constituents)
    news.extend(constituents)
    news.extend([str(dfe['randomDate'].iloc[0])])
    ascii_replace(filein, fileout, olds, news)
    
    ## modify adcprep and padcirc scripts
    # if hpc == 'hazel':
    olds0_hazel = ['ADCPREPADCPREP', 'XXX']
    news0_hazel = [f'{namerun}prep', '128']
    ascii_replace(pathtemplate/'adcprep.csh', pathout_sub/'adcprep.csh', olds0_hazel, news0_hazel)

    olds1_hazel = ['RUNRUN', 'NODESNODES']
    news1_hazel = [namerun, '128']
    ascii_replace(pathtemplate/'padcirc.csh', pathout_sub/'padcirc.csh', olds1_hazel, news1_hazel)

7it [00:00, 73.09it/s]


In [ ]:
## copy fort.13 fort.14 and executables
pathout = Path(r'/scratch/09536/tacuevas/02')
runs = [x for x in os.listdir(pathout) if x.isdigit()]

pathtemplate = pathout/'_template'
for r in runs:
    
    namerun = f'{r:04d}'
    pathout_sub = pathout/namerun
    filesIn = ['fort.13', 'fort.14', 'adcprep', 'padcirc']
    filesOut = ['fort.13', 'fort.14', 'adcprep', 'padcirc']

    for fi, fo in zip(filesIn, filesOut):
        shutil.copy(pathtemplate/fi, pathout_sub/fo)